In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns

In [7]:
# Reading input

articles = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/articles.csv')
# customers = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/customers.csv')
transactions_train = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv')

In [8]:
transactions_train['t_dat_format'] = pd.to_datetime(transactions_train['t_dat'])

transactions_train['month'] = transactions_train['t_dat_format'].dt.month

In [9]:
overall_top_products = transactions_train.\
                    groupby('article_id').\
                    agg({'t_dat':'count'}).reset_index().sort_values('t_dat', ascending=False).head(12)
aug_top_product = transactions_train[transactions_train['month']==8].\
                    groupby(['month','article_id']).\
                    agg({'t_dat':'count'}).reset_index().sort_values('t_dat', ascending=False).head(12)
sep_top_product = transactions_train[transactions_train['month']==9].\
                    groupby(['month','article_id']).\
                    agg({'t_dat':'count'}).reset_index().sort_values('t_dat', ascending=False).head(12)
oct_top_product = transactions_train[transactions_train['month']==10].\
                    groupby(['month','article_id']).\
                    agg({'t_dat':'count'}).reset_index().sort_values('t_dat', ascending=False).head(12)

oct_top_product = transactions_train[transactions_train['month']==10].\
                    groupby(['month','article_id']).\
                    agg({'t_dat':'count'}).reset_index().sort_values('t_dat', ascending=False).head(12)

In [10]:
## SEP vs overall common products
len(set(sep_top_product['article_id'].unique().tolist()) - set(overall_top_products['article_id'].unique().tolist()))

7

In [11]:
transactions_train.shape

(31788324, 7)

In [12]:
transactions_train = pd.merge(transactions_train,
                              articles[['article_id','index_group_name']].drop_duplicates(),
                              left_on = 'article_id',
                              right_on = 'article_id',
                              how = 'inner'
                             )

In [21]:
customer_family_profile = transactions_train[transactions_train['month'].isin([9,10])].groupby(['customer_id','index_group_name']).\
agg(unique_articles_count=('article_id','nunique')).reset_index()

In [24]:
customer_family_profile['article_count_rank'] = customer_family_profile.groupby('customer_id')['unique_articles_count'].rank(method='first', ascending=False)

In [27]:
customer_family_profile = customer_family_profile[customer_family_profile['article_count_rank']==1]\
[['customer_id','index_group_name']]

In [35]:
top_12_artc_ign = transactions_train[transactions_train['month'].isin([9,10])].groupby(['index_group_name','article_id']).agg(tran_count=('t_dat','count')).reset_index()

top_12_artc_ign['article_rank'] = top_12_artc_ign.groupby('index_group_name')['tran_count'].rank(method='first', ascending=False)

top_12_artc_ign = top_12_artc_ign[top_12_artc_ign ['article_rank'] <= 12]

top_12_artc_ign.head()

,index_group_name,article_id,tran_count,article_rank
221,Baby/Children,306847002,367,12.0
239,Baby/Children,309864002,698,2.0
2662,Baby/Children,552346001,466,7.0
2671,Baby/Children,552346014,544,3.0
2680,Baby/Children,552370002,759,1.0


In [36]:
top_12_artc_overall = transactions_train[transactions_train['month'].isin([9,10])].groupby(['article_id']).agg(tran_count=('t_dat','count')).reset_index()

top_12_artc_overall = top_12_artc_overall.sort_values('tran_count', ascending=False).head()

top_12_artc_overall.head()

,article_id,tran_count
44462,706016001,8127
36915,673677002,7834
1918,399223001,6872
1461,372860001,5607
13496,568601006,5598


In [38]:
submission = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/sample_submission.csv')\
[['customer_id']]

In [39]:
submission.shape

(1371980, 1)

In [41]:
submission = pd.merge(submission,
                                   customer_family_profile,
                                   left_on = ['customer_id'],
                                   right_on = ['customer_id'],
                                   how='left'
                                  )

In [43]:
top_12_artc_ign.head()

,index_group_name,article_id,tran_count,article_rank
221,Baby/Children,306847002,367,12.0
239,Baby/Children,309864002,698,2.0
2662,Baby/Children,552346001,466,7.0
2671,Baby/Children,552346014,544,3.0
2680,Baby/Children,552370002,759,1.0


In [44]:
submission = pd.merge(submission,
                       top_12_artc_ign[['index_group_name','article_id']],
                       left_on = ['index_group_name'],
                       right_on = ['index_group_name'],
                       how='left').drop('')

In [47]:
submission.agg({'customer_id':'nunique'})

customer_id    1371980
dtype: int64

In [46]:
submission.groupby('index_group_name').agg({'customer_id':'nunique'})

,customer_id
index_group_name,
Baby/Children,41795
Divided,152978
Ladieswear,410204
Menswear,31114
Sport,18934


In [49]:
submission['article_id'] = ' 0'+submission['article_id'].astype('str')

In [ ]:
submission_df = submission.groupby('')

In [ ]:
transactions_train_filt['purchase_cycle_days'] = transactions_train_filt['purchase_cycle'].astype('str').str.split(' ', expand=True)[0]

In [ ]:
transactions_train_filt['next_purchase_date'] = transactions_train_filt['product_last_purchase_date'] + transactions_train_filt['purchase_cycle_days'].astype(int).map(timedelta)

In [ ]:
start_date = transactions_train_filt['t_dat_datetime'].max() + timedelta(-90)
end_date = transactions_train_filt['t_dat_datetime'].max() + timedelta(90)

In [ ]:
datetime.date(transactions_train_filt['t_dat_datetime'].max()) 

In [ ]:
start_date, end_date

In [ ]:
len(transactions_train_filt['customer_id'].unique())

In [ ]:
customer_next_purchase = transactions_train_filt[transactions_train_filt['next_purchase_date'].\
                        between(start_date, end_date, inclusive="both")]\
[['customer_id','t_dat','article_id']].drop_duplicates()

In [ ]:
customer_next_purchase.shape

In [ ]:
customer_next_purchase['repeat_purchase'] = customer_next_purchase.groupby(['customer_id'])['article_id'].transform('count')

In [ ]:
customer_next_purchase['product_imp_rank'] = customer_next_purchase.groupby(['customer_id'])['repeat_purchase'].rank(method="first",ascending=True)

In [ ]:
customer_next_purchase['article_id'] = ' 0'+customer_next_purchase['article_id'].astype('str')

In [ ]:
top_12_products = pd.DataFrame(customer_next_purchase[customer_next_purchase['product_imp_rank']<=12]\
                               [['customer_id','article_id']].drop_duplicates().\
                               groupby(['customer_id']).article_id.sum().reset_index())

In [ ]:
sample_submission = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/sample_submission.csv')\
[['customer_id']]

In [ ]:
sample_submission = pd.merge(sample_submission,
                             customer_next_purchase[customer_next_purchase['product_imp_rank']<=12]\
                             [['customer_id','article_id']].\
                             drop_duplicates(),
                             left_on = ['customer_id'],
                             right_on = ['customer_id'],
                             how = 'left'
                            )

In [ ]:
sample_submission.shape

In [ ]:
no_products_cust = sample_submission[sample_submission['article_id'].isna()]
atleast_1_products_cust = sample_submission.dropna(axis=0)

no_products_cust.shape, atleast_1_products_cust.shape

In [ ]:
# top_12_products = pd.DataFrame(atleast_1_products_cust.groupby(['customer_id']).article_id.sum().reset_index())

In [ ]:
top_12_repeat_purchase_products = transactions_train_filt.groupby('article_id').agg({'customer_id':"nunique"}).\
sort_values('customer_id', ascending=False).reset_index().head(12)[['article_id']]

top_12_repeat_purchase_products['article_id'] = ' 0'+top_12_repeat_purchase_products['article_id'].astype('str')

In [ ]:
# no_products_cust['article_id'].fillna( "0706016001 0706016002 0372860001 0399223001 0464297007 0160442007 0610776002 0156231001 0399256001 0372860002 0706016003 0759871002", inplace=True)
# no_products_cust.shape
no_products_cust['article_id'].fillna( top_12_repeat_purchase_products.article_id.sum(), inplace=True)
no_products_cust.shape

In [ ]:
atleast_1_products_cust_full = pd.DataFrame({'customer_id':[],'article_id':[]})
atleast_1_products_cust_full

for customer_id in atleast_1_products_cust['customer_id'].unique().tolist():
    customer_id_list = []
    article_id_list = []
    cust_products = atleast_1_products_cust[atleast_1_products_cust['customer_id'] == customer_id]\
    ['article_id'].unique().tolist()
    
    if len(cust_products) < 12:
        product_to_add = list(set(top_12_repeat_purchase_products['article_id'].unique().tolist()) -\
                              set (cust_products))
        
        for article_id in product_to_add[0: (12 - len(cust_products))]:
            customer_id_list.append(customer_id)
            article_id_list.append(article_id)
            
        extra_products = pd.DataFrame({"customer_id":customer_id_list,'article_id':article_id_list})
            
        atleast_1_products_cust_full = pd.concat([atleast_1_products_cust_full, extra_products], axis = 0)
    
    atleast_1_products_cust_full = pd.concat([atleast_1_products_cust_full, 
                                             atleast_1_products_cust[atleast_1_products_cust['customer_id'] == customer_id]]
                                             , axis = 0)

In [ ]:
final_submisison.to_csv('submission.csv', index =False)

In [ ]:
sample_df = atleast_1_products_cust[:12]

In [ ]:
cust_list = sample_df['customer_id'].unique().tolist()

In [ ]:
for i in cust_list:
    [sample_df['customer_id'] == i]['article_id'].unique().tolist()
    

In [ ]:
(sample_df[sample_df['customer_id'] == '00009d946eec3ea54add5ba56d5210ea898def4b46c68570cf0096d962cacc75']\
['article_id'].unique().tolist()[0].strip())

In [ ]:
product_list = ["0706016001", "0706016002",
                "0372860001", '0399223001', '0464297007', '0160442007', 
                '0610776002', '0156231001', '0399256001', '0372860002',
                '0706016003', '0759871002']

In [ ]:
for i in product_list